In [1]:
#shakespeare texts
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-02-15 16:24:36--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8001::154, 2606:50c0:8003::154, 2606:50c0:8000::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8001::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: 'input.txt'

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.07s   

2023-02-15 16:24:36 (15.9 MB/s) - 'input.txt' saved [1115394/1115394]



In [5]:
#read shakespeare text:

with open('input.txt','r',encoding='utf-8') as file:
    text = file.read()

In [6]:
text[:1000]

"First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we become rakes: for the gods know I\nspeak this in hunger 

In [10]:
chars = sorted(list(set(text)))
print(chars)
print(''.join(chars))
print(len(chars))

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [11]:
#tokenization:

char_to_int = {char:index for index,char in enumerate(chars)}
int_to_char = {index:char for index,char in enumerate(chars)}

In [18]:
#mapping from chars to integers
def encode(string):
    integers = [char_to_int[char] for char in string]
    return integers

#mapping from integers to chars
def decode(integers):
    string =  ''.join([int_to_char[i] for i in integers])
    return string

print(encode('hii there'))
print(decode(encode('hii there')))

##Best practice - subword tokenizer

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [22]:
#tokenize entire shakespeare text
import torch
data = torch.tensor(encode(text))
data[:1000]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
        47, 59, 57,  1, 47, 57,  1, 41, 

In [309]:
split_size = 0.9

train_data = data[:int(len(data) * split_size)]
val_data = data[int(len(data) * split_size):]

In [32]:
len(train_data) + len(test_data)

1115394

In [42]:
block_size = 8 # length of the random chunk of text to put into transformer 
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [102]:
'''
x - is our context input variable
y - next character we are trying to predict based on x input variable
'''
x = train_data[:block_size]
y = train_data[1:block_size + 1]

for t in range(block_size):
    context = x[:t + 1]
    next_char = y[t]
    print('Context -> Target: ',context,' -> ',next_char)

Context -> Target:  tensor([18])  ->  tensor(47)
Context -> Target:  tensor([18, 47])  ->  tensor(56)
Context -> Target:  tensor([18, 47, 56])  ->  tensor(57)
Context -> Target:  tensor([18, 47, 56, 57])  ->  tensor(58)
Context -> Target:  tensor([18, 47, 56, 57, 58])  ->  tensor(1)
Context -> Target:  tensor([18, 47, 56, 57, 58,  1])  ->  tensor(15)
Context -> Target:  tensor([18, 47, 56, 57, 58,  1, 15])  ->  tensor(47)
Context -> Target:  tensor([18, 47, 56, 57, 58,  1, 15, 47])  ->  tensor(58)


In [351]:
### random N(batch_size) texts of size = block_size are stack into one batch

#batch dimension for block_size 
block_size = 8
batch_size = 4
num_batches = len(text) // batch_size

torch.manual_seed(1337)

'''
def create_batch(encoded_text):
    x_batch = torch.empty(batch_size,block_size)
    y_batch = torch.empty(batch_size,block_size)
    for i in range(batch_size):
        random_integer = torch.randint(high = len(encoded_text) - block_size,size = (1,))
        x_sample = encoded_text[random_integer:random_integer + block_size]
        y_sample = encoded_text[random_integer + 1:random_integer + block_size + 1]

        x_batch[i,:] = x_sample
        y_batch[i,:] = y_sample

    return x_batch,y_batch
'''

def create_batch(split):
    data = train_data if split == 'train' else  val_data      
    random_integers = torch.randint(high = len(data) - block_size,size = (batch_size,))
    x_batch = torch.stack([data[i:i + block_size] for i in random_integers])
    y_batch = torch.stack([data[i+1:i + block_size+1] for i in random_integers])
    return x_batch,y_batch

xb,yb = create_batch('train')
print(xb)
print(yb)

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b,:t+1]
        target  =yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53, 56, 1, 58, 46] the target: 39
when input is [44, 53, 56, 1

In [364]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)


class BigramLanguageModel(nn.Module):

    def __init__(self,vocab_size):
        super(BigramLanguageModel, self).__init__()
        self.vocab_size = vocab_size
        self.embedding = nn.Embedding(vocab_size,vocab_size)

    def forward(self,idx,targets):
        
        #idx and target are both (Batch,Time)
        logits = self.embedding(idx) #dimension: (Batch,Time,Channel). Time - block_size; Channel - vocab_size
        
        #reshape Logits and targets array according to Pytorch documentation of Cross-Entropy 
        B,T,C = logits.shape
        logits = logits.view(B * T,C)
        targets = targets.view(B * T)

        loss = F.cross_entropy(logits,targets)
        return logits, loss



In [365]:
bigramm = BigramLanguageModel(len(chars))
bigramm.forward(xb,yb)

(tensor([[-1.5101, -0.0948,  1.0927,  ..., -0.6126, -0.6597,  0.7624],
         [ 0.3323, -0.0872, -0.7470,  ..., -0.6716, -0.9572, -0.9594],
         [ 0.2475, -0.6349, -1.2909,  ...,  1.3064, -0.2256, -1.8305],
         ...,
         [-2.1910, -0.7574,  1.9656,  ..., -0.3580,  0.8585, -0.6161],
         [ 0.5978, -0.0514, -0.0646,  ..., -1.4649, -2.0555,  1.8275],
         [-0.6787,  0.8662, -1.6433,  ...,  2.3671, -0.7775, -0.2586]],
        grad_fn=<ViewBackward0>),
 tensor(4.8786, grad_fn=<NllLossBackward0>))

In [347]:
embedding = nn.Embedding(len(chars),8)
embedding.forward()